In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("SparkSQL_Transformations") \
    .getOrCreate()

# Load the dataset
file_path = "/FileStore/tables/Loan/loan.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Register as SQL table
df.createOrReplaceTempView("loans")



In [0]:
# Sample SQL Queries Using Filter:
# Example 1. Filter loans with Income > 10,000
high_loans = spark.sql("SELECT * FROM loans WHERE `Income` > 10000")
high_loans.show()

# Example 1: Using Filter function Filter rows with Income > 10,000
high_loans_df = df.filter(df["Income"] > 2000)
high_loans_df.show()



+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+------------+----------------+------------------+
|Customer_ID|Age|Gender|         Occupation|Marital Status|Family Size|Income|Expenditure|Use Frequency|   Loan Category|Loan Amount|Overdue| Debt Record| Returned Cheque| Dishonour of Bill|
+-----------+---+------+-------------------+--------------+-----------+------+-----------+-------------+----------------+-----------+-------+------------+----------------+------------------+
|    IB14001| 30|  MALE|       BANK MANAGER|        SINGLE|          4| 50000|      22199|            6|         HOUSING| 10,00,000 |      5|      42,898|               6|                 9|
|    IB14008| 44|  MALE|          PROFESSOR|       MARRIED|          6| 51000|      19999|            4|        SHOPPING|     50,000|      3|      33,999|               1|                 5|
|    IB14012| 30|FEMALE|            DENTIST| 

In [0]:
# Sample SQL Queries Using Aggregation:
# 1. Aggregation: Calculate the average Income
avg_loan = spark.sql("SELECT AVG(Income) AS Avg_Income FROM loans")
avg_loan.show()


+-----------------+
|       Avg_Income|
+-----------------+
|68339.49145299145|
+-----------------+



In [0]:
# Sample SQL Queries Using GroupBy:
# 1. Grouping by Occupation and calculating the average Income for each occupation
# SQL Query to group by 'Occupation' and calculate average 'Income'
group_by_occupation_query = """
SELECT 
    Occupation, 
    AVG(Income) AS Avg_Income
FROM customer_info
GROUP BY Occupation
"""
group_by_occupation_result = spark.sql(group_by_occupation_query)
group_by_occupation_result.show()

#2. Grouping by Gender and calculating the total Expenditure for each gender
group_by_gender_query = """
SELECT 
    Gender, 
    SUM(Expenditure) AS Total_Expenditure
FROM customer_info
GROUP BY Gender
"""

group_by_gender_result = spark.sql(group_by_gender_query)
group_by_gender_result.show()



+----------+----------+
|Occupation|Avg_Income|
+----------+----------+
|  Engineer|   40000.0|
|   Teacher|   45000.0|
|    Doctor|   50000.0|
|    Lawyer|   60000.0|
| Architect|   55000.0|
+----------+----------+

+------+-----------------+
|Gender|Total_Expenditure|
+------+-----------------+
|  Male|           105000|
|Female|            65000|
+------+-----------------+



In [0]:
# Sample SQL Queries Using Joins:
# Create sample data for customer_info (5 records)
customer_data = [
    ("C001", 30, "Male", "Engineer", "Married", 3, 40000, 30000, 4, "Good", 0, 0),
    ("C002", 40, "Female", "Teacher", "Single", 2, 45000, 20000, 5, "Fair", 1, 1),
    ("C003", 35, "Male", "Doctor", "Married", 4, 50000, 35000, 6, "Excellent", 0, 0),
    ("C004", 50, "Female", "Lawyer", "Married", 5, 60000, 45000, 3, "Good", 0, 0),
    ("C005", 45, "Male", "Architect", "Single", 3, 55000, 40000, 7, "Fair", 1, 1)
]

# Create DataFrame for customer_info
customer_columns = ["Customer_ID", "Age", "Gender", "Occupation", "Marital Status", "Family Size", 
                    "Income", "Expenditure", "Use Frequency", "Debt Record", "Returned Cheque", "Dishonour of Bill"]
customer_info_df = spark.createDataFrame(customer_data, customer_columns)

# Create sample data for loan_info (5 records)
loan_data = [
    ("C001", "Personal", "50000", 1),
    ("C002", "Auto", "20000", 0),
    ("C003", "Home", "150000", 2),
    ("C004", "Education", "25000", 0),
    ("C005", "Personal", "100000", 1)
]

# Create DataFrame for loan_info
loan_columns = ["Customer_ID", "Loan Category", "Loan Amount", "Overdue"]
loan_info_df = spark.createDataFrame(loan_data, loan_columns)

# Register both DataFrames as temp views (tables)
customer_info_df.createOrReplaceTempView("customer_info")
loan_info_df.createOrReplaceTempView("loans")

# Inner Join: Get the customer details along with their loan information
inner_join_query = """
SELECT 
    c.Customer_ID, 
    c.Age, 
    c.Gender, 
    c.Occupation, 
    c.Income, 
    l.`Loan Category`, 
    l.`Loan Amount`, 
    l.Overdue
FROM customer_info c
INNER JOIN loans l 
ON c.Customer_ID = l.Customer_ID
"""

inner_join_result = spark.sql(inner_join_query)
inner_join_result.show()

# Left Join: Get all customers and their loan information (if available)
left_join_query = """
SELECT 
    c.Customer_ID, 
    c.Age, 
    c.Gender, 
    c.Occupation, 
    c.Income, 
    l.`Loan Category`, 
    l.`Loan Amount`, 
    l.Overdue
FROM customer_info c
LEFT JOIN loans l 
ON c.Customer_ID = l.Customer_ID
"""

left_join_result = spark.sql(left_join_query)
left_join_result.show()




+-----------+---+------+----------+------+-------------+-----------+-------+
|Customer_ID|Age|Gender|Occupation|Income|Loan Category|Loan Amount|Overdue|
+-----------+---+------+----------+------+-------------+-----------+-------+
|       C001| 30|  Male|  Engineer| 40000|     Personal|      50000|      1|
|       C002| 40|Female|   Teacher| 45000|         Auto|      20000|      0|
|       C003| 35|  Male|    Doctor| 50000|         Home|     150000|      2|
|       C004| 50|Female|    Lawyer| 60000|    Education|      25000|      0|
|       C005| 45|  Male| Architect| 55000|     Personal|     100000|      1|
+-----------+---+------+----------+------+-------------+-----------+-------+

+-----------+---+------+----------+------+-------------+-----------+-------+
|Customer_ID|Age|Gender|Occupation|Income|Loan Category|Loan Amount|Overdue|
+-----------+---+------+----------+------+-------------+-----------+-------+
|       C001| 30|  Male|  Engineer| 40000|     Personal|      50000|      1